In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder.appName("sparkdev-stream-json demo").master("local[*]").getOrCreate()

23/04/09 19:19:32 WARN Utils: Your hostname, Pavans-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.29.143 instead (on interface en0)
23/04/09 19:19:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/09 19:19:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
input_schema = StructType([
    StructField(name="registration_dttm", dataType=StringType(), nullable=True),
    StructField(name="id", dataType=IntegerType(), nullable=True),
    StructField(name="first_name", dataType=StringType(), nullable=True),
    StructField(name="last_name", dataType=StringType(), nullable=True),
    StructField(name="email", dataType=StringType(), nullable=True),
    StructField(name="gender", dataType=StringType(), nullable=True),
    StructField(name="ip_address", dataType=StringType(), nullable=True),
    StructField(name="cc", dataType=StringType(), nullable=True),
    StructField(name="country", dataType=StringType(), nullable=True),
    StructField(name="birthdate", dataType=StringType(), nullable=True),
    StructField(name="salary", dataType=DoubleType(), nullable=True),
    StructField(name="title", dataType=StringType(), nullable=True),
    StructField(name="comments", dataType=StringType(), nullable=True)
])

In [5]:
stream_df = spark.readStream.format("json").schema(input_schema).option("path", "input_data/json").load()

In [6]:
stream_count_df = stream_df.groupBy(col("country")).count().orderBy("count", ascending=False).limit(10)

In [7]:
write_stream_query = stream_count_df \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("checkpointLocation", "streaming-checkpoint-loc-json") \
    .trigger(processingTime="10 second") \
    .start()

23/04/09 19:28:27 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [8]:
write_stream_query.awaitTermination()

print("Streaming Application Completed.")

-------------------------------------------
Batch: 0
-------------------------------------------
+-----------+-----+
|    country|count|
+-----------+-----+
|      China|  189|
|  Indonesia|   97|
|     Russia|   62|
|Philippines|   45|
|   Portugal|   38|
|     Brazil|   38|
|     France|   37|
|     Poland|   35|
|     Sweden|   25|
|      Japan|   20|
+-----------+-----+

23/04/09 19:31:06 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 26320 milliseconds


-------------------------------------------
Batch: 1
-------------------------------------------
+-------------+-----+
|      country|count|
+-------------+-----+
|        China|  362|
|    Indonesia|  186|
|       Russia|  123|
|  Philippines|   91|
|     Portugal|   79|
|       Poland|   76|
|       Brazil|   71|
|       France|   70|
|       Sweden|   60|
|United States|   40|
+-------------+-----+

23/04/09 19:31:49 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 19877 milliseconds


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/pavanmantha/Library/Application Support/JetBrains/DataSpell2022.3/projects/workspace/venv/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/pavanmantha/Library/Application Support/JetBrains/DataSpell2022.3/projects/workspace/venv/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 